## scikit-learn sample_weight compliance report

This notebook runs compliance tests on all scikit-learn estimators. Estimator as inspected to check whether they are expected to have a stochastic fit or not. If the fit is stochastic, a dedicated statistical test is performed, otherwise a deterministic estimator check is run instead.

In [1]:
import sklearn

sklearn.show_versions()


System:
    python: 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:13:44) [Clang 16.0.6 ]
executable: /Users/shrutinath/micromamba/envs/scikit-learn/bin/python
   machine: macOS-14.3-arm64-arm-64bit

Python dependencies:
      sklearn: 1.7.dev0
          pip: 24.0
   setuptools: 75.8.0
        numpy: 2.0.0
        scipy: 1.14.0
       Cython: 3.0.10
       pandas: 2.2.2
   matplotlib: 3.9.0
       joblib: 1.4.2
threadpoolctl: 3.5.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: openblas
    num_threads: 8
         prefix: libopenblas
       filepath: /Users/shrutinath/micromamba/envs/scikit-learn/lib/libopenblas.0.dylib
        version: 0.3.27
threading_layer: openmp
   architecture: VORTEX

       user_api: openmp
   internal_api: openmp
    num_threads: 8
         prefix: libomp
       filepath: /Users/shrutinath/micromamba/envs/scikit-learn/lib/libomp.dylib
        version: None


In [3]:
from inspect import signature
import traceback
import warnings
import pandas as pd
from sklearn.utils import all_estimators
from sklearn.exceptions import ConvergenceWarning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import Ridge
import threadpoolctl

from sample_weight_audit import check_weighted_repeated_estimator_fit_equivalence
from sample_weight_audit.sklearn_stochastic_params import STOCHASTIC_FIT_PARAMS

# HistGradientBoostingClassifier trashes the OpenMP thread pool on repeated
# small fits.
threadpoolctl.threadpool_limits(limits=1, user_api="openmp")
warnings.filterwarnings("ignore", category=RuntimeWarning)  # division by zero in AdaBoost
warnings.filterwarnings("ignore", category=ConvergenceWarning)  # liblinear can fail to converge
warnings.filterwarnings("ignore", category=UserWarning)  # KBinsDiscretizer with collapsed bins

In [4]:
from sklearn.linear_model import LogisticRegressionCV

ESTIMATORS_TO_SKIP = [
    LogisticRegressionCV,  # too slow and already somewhat tested by LogisticRegression
]

In [5]:
N_STOCHASTIC_FITS = 100
N_STATISTICAL_TESTS = 58  # measured by running the script
BONFERRONI_CORRECTION = 1 / N_STATISTICAL_TESTS
TEST_THRESHOLD = 0.05 * BONFERRONI_CORRECTION


statistical_test_results = []
missing_sample_weight_support = []
errors = []


for est_name, est_class in all_estimators(
    type_filter=["classifier", "regressor", "cluster", "transformer"]
):
    if est_class in ESTIMATORS_TO_SKIP:
        print(f"Skipping {est_name}")
        continue

    if "sample_weight" not in signature(est_class.fit).parameters:
        print(f"⚠ {est_name} does not support sample_weight")
        missing_sample_weight_support.append(est_name)
        continue

    try:
        if est_name == "CategoricalNB":
            # This estimator expects ordinal inputs so we need to discretize the input
            # features. This is not really valid but it's the best we can do while
            # keeping the dataset generation process common to all estimators of a
            # given type.
            est = Pipeline(
                steps=[
                    (
                        "kbinsdiscretizer",
                        KBinsDiscretizer(
                            encode="ordinal", quantile_method="averaged_inverted_cdf"
                        ),
                    ),
                    ("est", est_class(**STOCHASTIC_FIT_PARAMS.get(est_class, {}))),
                ]
            )
        else:
            est = est_class(**STOCHASTIC_FIT_PARAMS.get(est_class, {}))
    except TypeError as e:
        print(f"⚠ {est_name} failed to instantiate: {e}")
        continue

    print(f"Evaluating {est}")
    try:
        result = check_weighted_repeated_estimator_fit_equivalence(
            est,
            est_name,
            test_name="kstest",
            n_stochastic_fits=N_STOCHASTIC_FITS,
            random_state=0,
        )
        pass_or_fail = "✅" if result.pvalue > TEST_THRESHOLD else "❌"
        print(f"{pass_or_fail} {est_name}: (pvalue: {result.pvalue:.3f})")
        statistical_test_results.append(result)
         # Add extra test with KMeans pipelines to Ridge
        if est_name in ["MiniBatchKMeans","KMeans","BisectingKMeans"]:
            est = Pipeline(
                steps=[
                    (
                        "est",
                        est,
                    ),
                    ("ridge", Ridge(fit_intercept=False)),
                ]
            )  
            est_name = est_name+"_pipelined"
            result = check_weighted_repeated_estimator_fit_equivalence(
                est,
                est_name,
                test_name="kstest",
                n_stochastic_fits=N_STOCHASTIC_FITS,
                random_state=0,
            )
            pass_or_fail = "✅" if result.pvalue > TEST_THRESHOLD else "❌"
            print(f"{pass_or_fail} {est_name}: (pvalue: {result.pvalue:.3f})")
            statistical_test_results.append(result)
        
    except Exception as e:
        print(f"❌ {est} error with: {e}")
        errors.append((est, e))

results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

⚠ ARDRegression does not support sample_weight
Evaluating AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                    min_weight_fraction_leaf=0.1))


100%|██████████| 100/100 [00:05<00:00, 16.90it/s]


✅ AdaBoostClassifier: (pvalue: 0.815)


100%|██████████| 100/100 [00:05<00:00, 17.27it/s]


✅ AdaBoostClassifier_pipelined: (pvalue: 0.815)
Evaluating AdaBoostRegressor(estimator=DecisionTreeRegressor(max_features=0.5,
                                                  min_weight_fraction_leaf=0.1))


...

❌ RANSACRegressor() error with: Weights sum to zero, can't be normalized
⚠ RBFSampler does not support sample_weight
⚠ RFE does not support sample_weight
⚠ RFECV does not support sample_weight
⚠ RadiusNeighborsClassifier does not support sample_weight
⚠ RadiusNeighborsRegressor does not support sample_weight
⚠ RadiusNeighborsTransformer does not support sample_weight
Evaluating RandomForestClassifier()


100%|██████████| 100/100 [00:09<00:00, 10.50it/s]


❌ RandomForestClassifier: (pvalue: 0.000)


 24%|██▍       | 24/100 [00:02<00:07,  9.85it/s]


KeyboardInterrupt: 

In [5]:
print(
    f"✅ {len([r for r in statistical_test_results if r.pvalue > TEST_THRESHOLD])} "
    "passed the statistical test"
)
print(
    f"❌ {len([r for r in statistical_test_results if r.pvalue <= TEST_THRESHOLD])} "
    "failed the statistical test"
)
print(f"❌ {len(errors)} other errors")
print(
    f"⚠ {len(missing_sample_weight_support)} estimators lack sample_weight "
    "support"
)
results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

✅ 45 passed the statistical test
❌ 14 failed the statistical test
❌ 3 other errors
⚠ 112 estimators lack sample_weight support


## Details on the statistical test results

In [6]:
results_df.sort_values("pvalue")[["estimator_name", "pvalue", "deterministic_predictions"]]

,estimator_name,pvalue,deterministic_predictions
42,NuSVC,2.208761e-59,False
48,RandomForestRegressor,2.208761e-59,False
54,SVC,2.208761e-59,False
47,RandomForestClassifier,4.417521e-57,False
2,BaggingClassifier,4.395434e-55,False
25,HistGradientBoostingClassifier,5.600644e-50,False
53,SGDRegressor,2.596277e-44,False
35,LinearSVC,6.314162e-19,False
26,HistGradientBoostingRegressor,2.708443e-18,False
3,BaggingRegressor,4.528308e-17,False


## Details on errors

In [7]:
import sys

for est, e in errors:
    print(f"❌ {est}: {e}")
    traceback.print_exception(e, file=sys.stdout)
    print()

❌ RANSACRegressor(): Weights sum to zero, can't be normalized
Traceback (most recent call last):
  File "/var/folders/_y/lfnx34p13w3_sr2k12bjb05w0000gn/T/ipykernel_72266/2504969921.py", line 49, in <module>
    result = check_weighted_repeated_estimator_fit_equivalence(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ogrisel/code/sample-weight-audit-nondet/src/sample_weight_audit/estimator_check.py", line 90, in check_weighted_repeated_estimator_fit_equivalence
    multifit_over_weighted_and_repeated(
  File "/Users/ogrisel/code/sample-weight-audit-nondet/src/sample_weight_audit/estimator_check.py", line 312, in multifit_over_weighted_and_repeated
    est_weighted = check_pipeline_and_fit(
                   ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ogrisel/code/sample-weight-audit-nondet/src/sample_weight_audit/estimator_check.py", line 224, in check_pipeline_and_fit
    est = est.fit(X, y, sample_weight=sample_weight)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^

## List of estimators with missing sample_weight support

In [8]:
for est_name in missing_sample_weight_support:
    print(est_name)

ARDRegression
AdditiveChi2Sampler
AffinityPropagation
AgglomerativeClustering
BernoulliRBM
Binarizer
Birch
CCA
ClassifierChain
ColumnTransformer
DictVectorizer
DictionaryLearning
FactorAnalysis
FastICA
FeatureAgglomeration
FeatureHasher
FeatureUnion
FixedThresholdClassifier
FunctionTransformer
GaussianProcessClassifier
GaussianProcessRegressor
GaussianRandomProjection
GenericUnivariateSelect
HDBSCAN
HashingVectorizer
IncrementalPCA
Isomap
KNNImputer
KNeighborsClassifier
KNeighborsRegressor
KNeighborsTransformer
KernelCenterer
KernelPCA
LabelBinarizer
LabelEncoder
LabelPropagation
LabelSpreading
Lars
LarsCV
LassoLars
LassoLarsCV
LassoLarsIC
LatentDirichletAllocation
LinearDiscriminantAnalysis
LocallyLinearEmbedding
MaxAbsScaler
MeanShift
MinMaxScaler
MiniBatchDictionaryLearning
MiniBatchNMF
MiniBatchSparsePCA
MissingIndicator
MultiLabelBinarizer
MultiTaskElasticNet
MultiTaskElasticNetCV
MultiTaskLasso
MultiTaskLassoCV
NMF
NearestCentroid
NeighborhoodComponentsAnalysis
Normalizer
Nystroe

## Example interactive inspection of the predictions and scores of a given estimator

In [9]:
results = next(r for r in statistical_test_results if r.estimator_name == "GaussianNB")
results.predictions_repeated.shape

(1, 3000)

In [10]:
results.predictions_weighted.shape

(1, 3000)

In [11]:
import numpy as np
np.abs(results.predictions_repeated - results.predictions_weighted).max()

np.float64(3.9999992207384594e-10)

In [12]:
np.abs(results.scores_repeated - results.scores_weighted).max()

np.float64(3.2652280879119644e-10)

In [13]:
from scipy.stats import kstest
kstest(results.scores_repeated, results.scores_weighted)

KstestResult(statistic=np.float64(1.0), pvalue=np.float64(1.0), statistic_location=np.float64(3.262703314947884), statistic_sign=np.int8(1))